In [1]:
import pandas as pd

In [2]:
import collections
import os
import bz2
import csv
import json
import bisect
import numpy as np
import datetime
import pickle
import random
import seaborn as sns


In [3]:
def add_ancestor(ancestors, samples, graph):
    for s in samples:
        for n, v in graph[s]:
            if n not in ancestors:
                ancestors.add(n)


In [32]:
def sample_layer_nodes(graph, layer_dict, post_size, layer_sample=5, initial_samples=None):
    chosen = set()
    ancestors = set()
    if initial_samples:
        for s in initial_samples:
            chosen.add(s)
        add_ancestor(ancestors, chosen, graph)
    yset = list(layer_dict.keys())
    yset.sort(reverse=True)
    max_y = max(yset)
    print('ancestor: ',ancestors)
    for y in yset:
        print('layer : ', y)
        if y == max_y:
            candidates = [(post_size[s], s) for s in layer_dict[y]]
        else:
            candidates = [(post_size[s], s) for s in layer_dict[y] if s in ancestors]
        if len(candidates) == 0:
            print(y)
            continue
        candidates.sort(reverse=True)
        print(len(candidates))
        candidates = candidates[:50]
        probs = np.array([v[0] for v in candidates])
        probs = probs / np.sum(probs)
        count = sum([1 if s in chosen else 0 for s in layer_dict[y]])
        if layer_sample > count:
            samples = np.random.choice([v[1] for v in candidates],
                                       size=layer_sample - count, replace=False, p=probs)
            chosen |= set(samples)
            add_ancestor(ancestors, samples, graph)
    return chosen, ancestors


In [33]:
graph['AskReddit']

[('gadgets', 0.04),
 ('gaming', 0.07),
 ('entertainment', 0.1),
 ('business', 0.11),
 ('science', 0.12),
 ('programming', 0.15),
 ('politics', 0.22),
 ('reddit.com', 0.56)]

In [34]:
chosen, ancestors = sample_layer_nodes(graph, reverse_dict, post_size_dict, layer_sample=5)

ancestor:  set()
layer :  10
84


KeyError: 'gameflysocial'

In [5]:
datafile = pd.read_json('submissions_creator_common_graph_100',lines=True)

In [6]:
datafile['weight'] = datafile['common'].apply(lambda x : len(x)/100)

In [8]:
datafile = datafile.sort_values(['weight'])
datafile.head()

,common,s1,s1_start,s2,s2_start,weight
1328730,[antdude],illusions,1314054230,ShiaLabeouf,1357410751,0.01
1722416,[RightWingersSuck],DenverBroncos,1286580415,TrueProgressive,1376151471,0.01
1722417,[99red],Barca,1286885132,TrueProgressive,1376151471,0.01
1722418,[Aschebescher],thrashmetal,1289197100,TrueProgressive,1376151471,0.01
1722419,[PinkSlimeIsPeople],moderatepolitics,1289327856,TrueProgressive,1376151471,0.01


In [7]:
datafile['weight'] = datafile.common.apply(lambda x : len(x)/100)

In [24]:
print(len(datafile))
datafile_query = datafile.query('weight>=0.04')
print(len(datafile_query))

2657461
90317


In [25]:
datafile_query.reset_index(drop=True,inplace=True)

In [26]:
csv_d3 = datafile_query[["s2","s1",'weight']]

In [27]:
graph = {}
for i in csv_d3.values:
    if(i[0] in graph):
        temp = graph[i[0]]
        temp.append((i[1],i[2]))
        graph[i[0]] = temp
    else:
        graph[i[0]] = [(i[1],i[2])]


In [28]:
len(graph)

19616

In [14]:
layer = pd.read_csv('./community_graph/layer_year.csv')


In [15]:
post_size = pd.read_csv('./community_graph/size.csv')


In [17]:
layer.head()

,Unnamed: 0,subreddit,layer
0,0,worldnews,1
1,1,reddit.com,0
2,2,politics,0
3,3,science,0
4,4,entertainment,0


In [18]:
post_size_dict = {}
for i in post_size.values:
    post_size_dict[i[0]] = i[1]

In [19]:
layer_dict_1 = {}
for i in layer.values:
    layer_dict_1[i[1]] = i[2]

In [20]:
reverse_dict = {}
for k in layer_dict_1:
    v = layer_dict_1[k]
    if v not in reverse_dict:
        reverse_dict[v] = []
    reverse_dict[v].append(k)

In [22]:
len(reverse_dict),len(layer_dict_1),len(post_size_dict)

(11, 30215, 713734)

KeyError: 7610

In [10]:
csv_d3 = csv_d3.rename(index=str, columns={"s2": "child", "s1": "parent"})

In [29]:
len(csv_d3)

90317

In [32]:
csv_d3.columns= ['id','parent']

In [35]:
csv_d3.to_csv('data.csv',index=False)

In [30]:
len(csv_d3.parent.unique())

4875

In [12]:
parent_child = {}
for i in csv_d3.values:
    parent = i[1]
    child = i[0]
    if parent in parent_child:
        temp = parent_child[parent]
        temp.append(child)
        parent_child[parent] = temp
    else:
        parent_child[parent] = [child]

print(len(parent_child))

4875


In [13]:
len(parent_child['reddit.com'])

4647

In [14]:
graph = {}
nodes = set()
nodes.add('reddit.com')
while len(nodes) !=0:
    curr = nodes.pop()
    if(curr in parent_child):
        nodes |= set(parent_child[curr])
        graph[curr] = parent_child[curr]
print(len(graph))

3994


In [ ]:
import json
graph_json = json.dumps(graph)

In [18]:
csv_all = open('d3-new-15-all.csv','w')

In [16]:
import os

In [19]:
child = set()
for i in graph:
    for j in graph[i]:
        if j not in child:
            child.add(j)
        else:
            csv_all.write(j+','+i+'\n')
csv_all.close()

In [22]:
n = set()
for i in graph:
    n.add(i)
    for j in graph[i]:
            n.add(j)

In [26]:
child = set()
child_1 = set()
for i in graph:
    for j in graph[i]:
        if j not in child:
            child.add(j)
        else:
            child_1.add(j)
print(len(set(list(child_1)+list(child))))

18854


In [23]:
len(n)

18855

In [ ]:
import numpy as np
parents = np.setdiff1d(csv_d3.parent.unique(),csv_d3.child.unique())

In [ ]:
len(parents)

In [ ]:
root = ['root' for i in parents]
    

In [ ]:
len(root)

In [ ]:
d = {'child': parents, 'parent':root}
temp = pd.DataFrame(data=d)

In [ ]:
d3 = temp.append(csv_d3)

In [ ]:
len(d3)

In [ ]:
d3.drop_duplicates('child',inplace=True)

In [ ]:
d3.to_csv("d3-noduplicates.csv",sep=',')

In [ ]:
csv_d3.to_csv("d3.csv",sep=',')

In [ ]:
source = datafile_query.s1.unique()

In [ ]:
import numpy as np
print(len(source))
print(len(datafile_query.s2.unique()))
len(np.intersect1d(source,datafile_query.s2.unique()))

In [ ]:
import json
json_val = {}
for row in source[7:]:
    children= datafile_query.query('s1==\''+str(row)+'\'')['s2'].values
    child = [dict(name=pn) for pn in children]
    json_val["name"] = row
    json_val["children"] = child
    print(json.dumps(json_val))
    break

In [ ]:
data = {}
data['nodes'] = []
data['links'] = []
names = []

for row in datafile_query.values:
	s1 = row[1]
	s2 = row[3]
	weight = row[-1]
	if not s1  in names:
		node1 = {}
		node1['id'] = s1
		node1['group'] = row[2]
		names.append(s1)
		data['nodes'].append(node1)
	if not s2  in names:
		node2 = {}
		node2['id'] = s2
		node2['group'] = row[4]
		names.append(s2)
		data['nodes'].append(node2)
	link = {}
	link['source'] = s1
	link['target'] = s2
	link['value'] = weight
	data['links'].append(link)

In [ ]:
print(len(data['nodes']),len(data['links']))

In [ ]:
import json
with open('sample_graph.json', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
data['links'][:100]